# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
from bs4 import BeautifulSoup

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
df_stocks = pd.read_csv("sp_500_stocks.csv")
df_stocks['Ticker']

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 505, dtype: object

## Acquiring an API Token

In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = "https://sandbox.iexapis.com/stable"

In [16]:
url="https://iexcloud.io/docs/api/"
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")
print(soup.title.text)
for link in soup.find_all("a"):
    print("Inner Text: {}".format(link.text))
    print("Title: {}".format(link.get("title")))
    print("href: {}".format(link.get("href")))
    
# gdp_table = soup.find("table", attrs={"class": "wikitable"})
# gdp_table_data = gdp_table.tbody.find_all("tr")  # contains 2 rows

# # Get all the headings of Lists
# headings = []
# for td in gdp_table_data[0].find_all("td"):
#     # remove any newlines and extra spaces from left and right
#     headings.append(td.b.text.replace('\n', ' ').strip())

# print(headings)

IEX Cloud API | IEX
Inner Text: 








Title: None
href: #
Inner Text: 




Title: None
href: /
Inner Text: 

IEX Cloud logo












API

Title: None
href: /
Inner Text: Introduction
Title: None
href: #introduction
Inner Text: API Reference
Title: None
href: #api-reference
Inner Text: API Versioning
Title: None
href: #api-versioning
Inner Text: Deprecation
Title: None
href: #deprecation
Inner Text: Attribution
Title: None
href: #attribution
Inner Text: Authentication
Title: None
href: #authentication
Inner Text: OAuth / OIDC Beta
Title: None
href: #oauth-oidc-beta
Inner Text: OAuth Benefits
Title: None
href: #oauth-benefits
Inner Text: Batch Requests
Title: None
href: #batch-requests
Inner Text: Data Formats
Title: None
href: #data-formats
Inner Text: Disclaimers
Title: None
href: #disclaimers
Inner Text: Error Codes
Title: None
href: #error-codes
Inner Text: Filter results
Title: None
href: #filter-results
Inner Text: How Credits Work
Title: None
href: #how-credits-work
Inner Te

In [5]:
quote = api_url + f'/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
response = requests.get(quote)
if  (response.status_code == 200):
    data = response.json()
    df = pd.DataFrame(list(data.items()),columns = ['key','value'])
    print (df)
else :
    (print(response.status_code))    

                       key                              value
0                   symbol                               AAPL
1              companyName                          Apple Inc
2          primaryExchange  BATNS/GQL(SRAELGC DKNLS)AEME  OTA
3         calculationPrice                               tops
4                     open                                  0
5                 openTime                               None
6               openSource                           facfolii
7                    close                                  0
8                closeTime                               None
9              closeSource                           iaflciof
10                    high                                  0
11                highTime                               None
12              highSource                               None
13                     low                                  0
14                 lowTime                               None
15      

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
ticker = df_stocks['Ticker']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
cols = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
df_info = pd.DataFrame(columns = cols)
df_info.append(pd.Series([symbol, price, market_cap, 'N\A'], index = cols), ignore_index = True)

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,AAPL,130.46,2262691936405,N\A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [11]:
final_dataframe = pd.DataFrame(columns = cols)
for symbol in df_stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = cols), 
                                        ignore_index = True)


In [12]:
final_dataframe

,Tickets,Stock Price,Market Capitalization,Number of shares to buy
0,A,149.26,45771427976,N/A
1,AAL,23.31,15465455644,N/A
2,AAP,200.24,13081444412,N/A
3,AAPL,133.03,2199932558504,N/A
4,ABBV,119.20,207765562941,N/A
...,...,...,...,...
500,YUM,122.20,35339767848,N/A
501,ZBH,159.24,34475055627,N/A
502,ZBRA,511.97,27507417157,N/A
503,ZION,54.06,9197916066,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [8]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [9]:
symbol_groups = list(chunks(df_stocks['Ticker'], 100))
symbol_strings = [] 
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
# print(symbol_strings)
final_dataframe = pd.DataFrame(columns = cols) 
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([symbol,
                                                            data[symbol]['quote']['latestPrice'],
                                                            data[symbol]['quote']['marketCap'],
                                                            'n/a'], 
                                                           index = cols
                                                          ), ignore_index = True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,150.740,44550461227,n/a
1,AAL,22.870,15033875238,n/a
2,AAP,203.890,13294035055,n/a
3,AAPL,135.039,2218638877505,n/a
4,ABBV,121.001,206045072008,n/a
...,...,...,...,...
500,YUM,120.658,36097287050,n/a
501,ZBH,162.502,33678977866,n/a
502,ZBRA,535.790,27797032482,n/a
503,ZION,56.200,9226899129,n/a


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [10]:
portfolio_value = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_value)
except ValueError:
    print ('Please enter the integer: ')
    
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe


Enter the value of your portfolio:  1000000


,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,150.740,44550461227,13
1,AAL,22.870,15033875238,86
2,AAP,203.890,13294035055,9
3,AAPL,135.039,2218638877505,14
4,ABBV,121.001,206045072008,16
...,...,...,...,...
500,YUM,120.658,36097287050,16
501,ZBH,162.502,33678977866,12
502,ZBRA,535.790,27797032482,3
503,ZION,56.200,9226899129,35


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()